<a href="https://colab.research.google.com/github/FangfangChen2023/ML-Projects/blob/main/GoodRead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import sklearn
import matplotlib.pyplot as plt
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import pydot
import graphviz
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Loading data

In [ ]:
# For loading the complete dataset
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c cs985-cs987-goodread-class-project

In [ ]:
! unzip /content/cs985-cs987-goodread-class-project.zip

Archive:  /content/cs985-cs987-goodread-class-project.zip
  inflating: my_goodreads_sample_submission.csv  
  inflating: my_goodreads_test.csv   
  inflating: my_goodreads_train.csv  


In [ ]:
# goodreads_train = pd.read_csv("my_goodreads_train.csv", error_bad_lines=False, engine="python")
goodreads_train = pd.read_csv("my_goodreads_train.csv")
goodreads_train.info()
# goodreads_train.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       700000 non-null  object
 1   book_id       700000 non-null  int64 
 2   review_id     700000 non-null  object
 3   rating        700000 non-null  int64 
 4   review_text   700000 non-null  object
 5   date_added    700000 non-null  object
 6   date_updated  700000 non-null  object
 7   read_at       628588 non-null  object
 8   started_at    486589 non-null  object
 9   n_votes       700000 non-null  int64 
 10  n_comments    700000 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 58.7+ MB


In [ ]:
# goodreads_test = pd.read_csv("my_goodreads_test.csv", error_bad_lines=False, engine="python")
goodreads_test = pd.read_csv("my_goodreads_test.csv")
goodreads_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    200000 non-null  int64 
 1   user_id       200000 non-null  object
 2   book_id       200000 non-null  int64 
 3   review_id     200000 non-null  object
 4   review_text   200000 non-null  object
 5   date_added    200000 non-null  object
 6   date_updated  200000 non-null  object
 7   read_at       179646 non-null  object
 8   started_at    139114 non-null  object
 9   n_votes       200000 non-null  int64 
 10  n_comments    200000 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 16.8+ MB


In [ ]:
goodreads_train.columns

Index(['user_id', 'book_id', 'review_id', 'rating', 'review_text',
       'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes',
       'n_comments'],
      dtype='object')

In [ ]:
train_data = goodreads_train.drop(["user_id", "book_id", "review_id", "date_added", "date_updated", "read_at","started_at"], axis=1)
test_data = goodreads_test.drop(["user_id", "book_id", "review_id", "date_added", "date_updated", "read_at","started_at"], axis=1)

In [ ]:
test_data = test_data.dropna()
test_data.head()

,Unnamed: 0,review_text,n_votes,n_comments
0,2212,"3-3.5 stars \n While I liked it, I didn't love...",1,0
1,655187,"First, I must fangirl over the ending. *OMG FA...",0,0
2,483724,Somewhere between 4 and 5 - really loved this ...,1,0
3,322858,I'll admit it - I was not expecting to enjoy t...,0,0
4,393932,Book #25 of 2011 \n I didn't read the accompyi...,0,0


In [ ]:
train_data = train_data.dropna()
train_data.head()

,rating,review_text,n_votes,n_comments
0,5,I thought this was a very good historical fant...,5,6
1,5,I've had this book recommended to me more than...,0,0
2,5,Seriously! You go and throw that twist into th...,0,0
3,4,Gosh!!! I'm still not recovering from the effi...,5,0
4,3,"3.5 stars rounded down. I can't help it, I jus...",0,0


# Cleaning the text data



In [ ]:
# Lowercase the review text
train_review = train_data["review_text"].apply(lambda x: str(x).lower())
test_review = test_data["review_text"].apply(lambda x: str(x).lower())

In [ ]:
# Remove the line breaks and extra spaces
train_review = train_review.apply(lambda x: " ".join(x.split()))
test_review = test_review.apply(lambda x: " ".join(x.split()))

In [ ]:
import re

# Removing https links from the text
train_review = train_review.apply(
    lambda x: re.sub(
        r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '',
        x, flags=re.MULTILINE))

test_review = test_review.apply(
    lambda x: re.sub(
        r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '',
        x, flags=re.MULTILINE))

In [ ]:
# Remove special charaters from the review text
train_review = train_review.apply(lambda x: re.sub('\W+',' ', x))
test_review = test_review.apply(lambda x: re.sub('\W+',' ', x))

In [ ]:
# Import nltk and download stopwords
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

# Load the stop words
stop_words = list(stopwords.words('english'))

# View the count of stop_words
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [ ]:
# Remove stop_words from reviews
train_review = train_review.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
test_review = test_review.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [ ]:
# Remove repeated words from reviews
train_review = train_review.apply(lambda x: ' '.join(set(x.split())))
test_review = test_review.apply(lambda x: ' '.join(set(x.split())))

# Building Model



## Two baseline and one deep NN models

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


## Embedding layer



### Training data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/goodread-class-project/my_goodreads_train.csv',usecols=['review_text','rating'],nrows=300000)
data.head(3)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the sentence
sentence = data['review_text']

# Define the pre-trained GloVe embeddings
embedding_dim = 50
# glove_file = "path/to/glove.6B.100d.txt"
# glove_file = "/content/drive/MyDrive/Goodread/glove.6B.100d.txt"
glove_file = "/content/drive/MyDrive/goodread-class-project/glove.6B.50d.txt"

# Tokenize the sentence
tokenizer = Tokenizer()
# tokenizer.fit_on_texts([sentence])
tokenizer.fit_on_texts(data['review_text'])
word_index = tokenizer.word_index
# sequences = tokenizer.texts_to_sequences([sentence])
sequences = tokenizer.texts_to_sequences(data['review_text'])

# Pad the sequence to a fixed length
# max_length = 10
max_length = 50
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Load the pre-trained GloVe embeddings
embeddings_index = {}
with open(glove_file) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create the embedding layer
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = tf.keras.layers.Embedding(len(word_index) + 1,
                                            embedding_dim,
                                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                            trainable=False)

# Pass the padded sequence through the embedding layer
embedded_sequence = embedding_layer(padded_sequences)

# Flatten the embedding to a 1D vector
embedded_vector = tf.keras.layers.Flatten()(embedded_sequence)

# Convert the embedded vector to a numpy array
embedded_array = embedded_vector.numpy()

# print(embedded_array)


### Testing data

In [ ]:
data_test = pd.read_csv('/content/drive/MyDrive/goodread-class-project/my_goodreads_test.csv',usecols=['review_text'])
data_test.head(3)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the sentence
sentence = data_test['review_text']

# Define the pre-trained GloVe embeddings
embedding_dim = 50
# glove_file = "path/to/glove.6B.100d.txt"
# glove_file = "/content/drive/MyDrive/Goodread/glove.6B.100d.txt"
glove_file = "/content/drive/MyDrive/goodread-class-project/glove.6B.50d.txt"

# Tokenize the sentence
tokenizer = Tokenizer()
# tokenizer.fit_on_texts([sentence])
tokenizer.fit_on_texts(data_test['review_text'])
word_index = tokenizer.word_index
# sequences = tokenizer.texts_to_sequences([sentence])
sequences = tokenizer.texts_to_sequences(data_test['review_text'])

# Pad the sequence to a fixed length
# max_length = 10
max_length = 50
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Load the pre-trained GloVe embeddings
embeddings_index = {}
with open(glove_file) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create the embedding layer
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = tf.keras.layers.Embedding(len(word_index) + 1,
                                            embedding_dim,
                                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                            trainable=False)

# Pass the padded sequence through the embedding layer
embedded_sequence = embedding_layer(padded_sequences)

# Flatten the embedding to a 1D vector
embedded_vector = tf.keras.layers.Flatten()(embedded_sequence)

# Convert the embedded vector to a numpy array
embedded_array = embedded_vector.numpy()

# print(embedded_array)

In [ ]:
test = pd.DataFrame(embedded_array)

X = df.drop(['rating'],axis=1)
y = df['rating']

y = pd.get_dummies(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## First NN Model


In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(2500,)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)

model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=256, callbacks=[early_stop])

## Second NN Model

In [ ]:
%%time
model = Sequential()

# Adding the first hidden layer with 256 units, using the relu activation function and L2 regularization
model.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_dim=2500))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Adding the second hidden layer with 128 units, using the relu activation function and L2 regularization
model.add(Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Adding the second hidden layer with 128 units, using the relu activation function and L2 regularization
model.add(Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Adding the output layer with 5 units and softmax activation function
model.add(Dense(units=5, activation='softmax'))

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Defining early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Fitting the model
history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stop])

# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

## Third NN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import numpy as np

# Define the learning rate scheduler
def lr_scheduler(epoch):
    lr = 0.001
    if epoch > 50:
        lr *= np.exp(-0.1)
    return lr

# Initialize the model
model = Sequential()

# Add the first hidden layer with 1024 units, using the tanh activation function and L2 regularization
model.add(Dense(units=1024, activation='tanh', kernel_regularizer=regularizers.l2(0.01), input_dim=2500))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Add the second hidden layer with 512 units, using the sigmoid activation function and dropout regularization
model.add(Dense(units=512, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Add the third hidden layer with 256 units, using the relu activation function and batch normalization
model.add(Dense(units=256, activation='relu'))
model.add(BatchNormalization())

# Add the fourth hidden layer with 128 units, using the tanh activation function and L2 regularization
model.add(Dense(units=128, activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Add the output layer with 5 units and softmax activation function
model.add(Dense(units=5, activation='softmax'))

# Compile the model with the RMSprop optimizer and learning rate scheduler
opt = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Define early stopping and learning rate scheduler callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
lr_scheduler = LearningRateScheduler(lr_scheduler, verbose=1)

# Fit the model
history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stop, lr_scheduler])


## LSTM


### Data preparation



In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data["review_text"])

x_train_seq = tokenizer.texts_to_sequences(train_data["review_text"])
x_test_seq = tokenizer.texts_to_sequences(test_data["review_text"])

In [ ]:
from keras.utils import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=100, padding='post')
x_train_pad

array([[  17,    5,  523, ...,   49, 1000,   24],
       [ 159,   55,   12, ...,    0,    0,    0],
       [ 441,   22,  148, ...,    0,    0,    0],
       ...,
       [ 620,  396,  223, ...,    0,    0,    0],
       [3629,   61,    1, ...,   15,    1,  177],
       [9354,    4,  154, ...,  259,  360, 2060]], dtype=int32)

In [ ]:
x_test_pad = pad_sequences(x_test_seq, maxlen=100, padding='post', truncating='post')
x_test_pad

array([[ 191,  191,  136, ...,    2,    1,  517],
       [  66,    3,  360, ...,   13, 3409,  298],
       [1285,  146,  223, ...,    0,    0,    0],
       ...,
       [3057,   30,   29, ...,    0,    0,    0],
       [   3,   36,  121, ...,    0,    0,    0],
       [  12,    9,    5, ...,    0,    0,    0]], dtype=int32)

In [ ]:
x_prepared = pd.DataFrame(x_train_pad)
# Because the output labels are one-hot vectors, so I also have to convert the lables to one-hot vectors before training the model.
from keras.utils import to_categorical
y_prepared = to_categorical(train_data["rating"]-1, num_classes=5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_mytest, y_train, y_mytest = train_test_split(x_prepared, y_prepared, test_size = 0.2, random_state = 42)

### model



In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras import optimizers
from keras.regularizers import l2
vocabulary_size = len(tokenizer.word_counts.keys())+1
max_words = 100
embedding_size = 100

model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
# model.add(LSTM(10, return_sequences=True))
model.add(LSTM(128, dropout=0.7, recurrent_dropout=0.7))
# model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))
model.add(Dropout(0.7))
# model.add(LSTM(128, dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(5, activation='softmax'))


In [ ]:
# Using adam as the optimizer and setting 0.0001 as the learning rate.
optimizer = optimizers.Adam(learning_rate = 0.001)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer= optimizer, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          31429200  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 5)                 645       
                                                                 
Total params: 31,547,093
Trainable params: 31,547,093
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# from keras.callbacks import EarlyStopping

# Define the early stopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights = True)


In [ ]:
# result = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, shuffle =True, callbacks=[early_stopping])
result = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, shuffle =True)

Epoch 1/2
 6363/14000 [============>.................] - ETA: 2:24:25 - loss: 1.3592 - accuracy: 0.3764

## Bert


### Data preparation




In [ ]:
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize the text data
tokenized_train = tokenizer.batch_encode_plus(train_review, padding=True, truncation=True, return_attention_mask=True, return_token_type_ids=True, return_tensors='pt')
tokenized_test = tokenizer.batch_encode_plus(test_review, padding=True, truncation=True, return_attention_mask=True, return_token_type_ids=True, return_tensors='pt')


In [ ]:
from keras.utils import to_categorical
labels = to_categorical(train_data["rating"]-1, num_classes=5)

import torch
labels = torch.tensor(labels)

from sklearn.model_selection import train_test_split

# train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(tokenized_train, labels, test_size=0.2, random_state=42)
train_input_ids, val_input_ids, train_attention_masks, val_attention_masks, train_labels, val_labels = train_test_split(tokenized_train['input_ids'], tokenized_train['attention_mask'], labels, test_size=0.2, random_state=42)


### model




In [ ]:
import tensorflow as tf
import transformers

# Load the pre-trained BERT model
bert_model = transformers.TFBertModel.from_pretrained('bert-base-uncased')

# Define the classification layer
classification_layer = tf.keras.layers.Dense(5, activation='softmax')

# Define the model architecture
input_ids = tf.keras.Input(shape=(512,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(512,), dtype=tf.int32, name='attention_mask')
bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]
classification_output = classification_layer(bert_output)
model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=classification_output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Convert PyTorch tensors to TensorFlow tensors
train_input_ids = tf.convert_to_tensor(train_input_ids.numpy())
val_input_ids = tf.convert_to_tensor(val_input_ids.numpy())
train_attention_masks = tf.convert_to_tensor(train_attention_masks.numpy())
val_attention_masks = tf.convert_to_tensor(val_attention_masks.numpy())

train_labels = tf.convert_to_tensor(train_labels.numpy())
val_labels = tf.convert_to_tensor(val_labels.numpy())

 model.fit(
    x=[train_input_ids, train_attention_masks], y=train_labels,
    epochs=5,
    batch_size=32,
    validation_data=([val_input_ids, val_attention_masks], val_labels)
)

### Result

In [ ]:
loss, accuracy = model.evaluate(X_mytest, y_mytest)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
review_id = goodreads_test['review_id']
result = model.predict(x_test_pad)
# pre_result = pd.DataFrame(data=result, columns=[ "rating"], index=review_id)
# pre_result.to_csv('prediction.csv')

In [ ]:

predicted_classes = np.argmax(result, axis=-1) + 1
predicted_classes

In [ ]:
pre_result = pd.DataFrame(data=predicted_classes, columns=[ "rating"], index=review_id)
review_id = pd.DataFrame(data=review_id, columns=[ "review_id"])
pre_result.join(other=review_id, lsuffix='_')

pre_result.to_csv('prediction.csv')